In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None 
#https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
import numpy as np
import re
import matplotlib.pyplot as plt

df = pd.read_csv('original_dataset.csv', low_memory = False, dtype = {'locality': str, 'postcode': str, 'property_subtype': str})
raw_df = df

In [3]:
df = df[['locality', 'postcode', 'property_subtype', 'price', 'rooms_number', 'area']]
regex = "(10|11|12)\d{2}"
df = df[df.postcode.str.match(regex, na=False)|df.locality.str.match(regex, na=False)]

In [5]:
df = df.drop(df[df['price'].isnull()].index)
df = df.drop(df[df['area'].isnull()].index)

df = df.drop(df[df['price'] == 'None'].index)
df = df.drop(df[df['price'] == '@0'].index)
df = df.drop(df[df['price'] == '1À000'].index)
df = df.drop(df[df['price'] == '2À000'].index)
df = df.drop(df[df['price'] == '3À000'].index)
df = df.drop(df[df['price'] == '23456789'].index)

df = df.drop(df[df['area'] == 'None'].index)
df = df.drop(df[df['area'] == 'Not specified'].index)

df = df.drop(df[df['locality'] == '1044'].index)
df = df.drop(df[df['locality'] == '1212'].index)

In [6]:
df['locality'] = df['locality'].str.upper()
df['property_subtype'] = df['property_subtype'].str.upper()

In [7]:
character_map = {
    'É' : 'E',
    '\(' : '', 
    '\)' : '', 
    '_':'-'
}
df.replace(character_map, inplace=True, regex=True)

In [8]:
conversions = {
    'FLAT-STUDIO':'FLAT',
    'SERVICE-FLAT':'FLAT'
}
df['property_subtype'] = df['property_subtype'].map(conversions).fillna(df['property_subtype'])

In [12]:
def extract_postcode(d): 
    return re.sub("((\d{4}){0,1})(([A-Za-z- ]+[0-9]{0,2}){0,1})", r'\2',d)
df['extracted_pc_from_loc'] = df[~df['locality'].isnull()]['locality'].apply(extract_postcode)
df['extracted_pc_from_pc'] = df[~df['postcode'].isnull()]['postcode'].apply(extract_postcode)

In [13]:
df['merged_postcode'] = df['extracted_pc_from_loc'].replace(np.nan, '') + df['extracted_pc_from_pc'].replace(np.nan, '')
def unmerge_postcode(d): 
    return re.sub("(\d{4}){0,1}(\d{4})$", r'\2', d)
df['unmerged_postcode'] = df[~df['merged_postcode'].isnull()]['merged_postcode'].apply(unmerge_postcode)

In [14]:
df = df.drop(['locality', 'postcode', 'extracted_pc_from_loc', 'extracted_pc_from_pc', 'merged_postcode'], axis=1, errors='ignore')

In [15]:
def clean_area(d): 
    return re.sub('^([0-9]+)( {0,}m{0,}²{0,})', r'\1',d)
df['area'] = df['area'].apply(clean_area)

In [16]:
df['area'] = pd.to_numeric(df['area'])

In [17]:
df = df.drop(df[df['area']>600].index)

In [18]:
def clean_price(d): 
    return re.sub("^(€{0,})(([0-9\,\.])+)(€{0,})$", r'\2',d)
df['price'] = df['price'].apply(clean_price)

In [19]:
character_map = {
    '\.' : '', 
    '\,' : ''
}
df.replace(character_map, inplace=True, regex=True)

In [20]:
df['price'] = pd.to_numeric(df['price'])
df['area'] = pd.to_numeric(df['area'])

In [21]:
df = df.drop(df[df['price']==max(df['price'])].index)

In [22]:
df['rooms_number'] = pd.to_numeric(df['rooms_number'], errors='coerce')

In [85]:
df = df.drop(df[(df['property_subtype']!='APARTMENT-BLOCK')&(df['rooms_number']>10)].index)

In [90]:
df.to_csv('transformed_dataset.csv', index=False)